# OpenAI Assitants API

The [Assistants API](https://platform.openai.com/docs/assistants/overview) allows you to build AI assistants within your own applications. An Assistant has instructions and can leverage models, tools, and knowledge to respond to user queries. The Assistants API currently supports three types of tools: Code Interpreter, Retrieval, and Function calling.

TruLens can be easily integrated with the assistants API to provide the same observability tooling you are used to when building with other frameworks.

## Set keys

In [1]:
import os
os.environ["OPENAI_API_KEY"] = "sk-..."

## Create the assistant

Let's create a new assistant that answers questions about the famous *Paul Graham Essay*.

The easiest way to get it is to download it via this link and save it in a folder called data. You can do so with the following command

In [2]:
!wget https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt -P data/

--2024-04-01 17:35:59--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘data/paul_graham_essay.txt.14’

paul_graham_essay.t 100%[===================>]  73.28K  --.-KB/s    in 0.03s   

2024-04-01 17:35:59 (2.20 MB/s) - ‘data/paul_graham_essay.txt.14’ saved [75042/75042]



In [3]:
from trulens_eval import Tru

tru = Tru()

tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


In [14]:
from openai import OpenAI
client = OpenAI()

# upload the file

file = client.files.create(
  file=open("data/paul_graham_essay.txt", "rb"),
  purpose='assistants'
)

# create the assistant with access to a retrieval tool

assistant = client.beta.assistants.create(
  name="Paul Graham Essay Assistant",
  instructions="You are an assistant that answers questions about Paul Graham.",
  tools=[{"type": "retrieval"}],
  model="gpt-4-turbo-preview",
  file_ids=[file.id]
)

In [15]:
type(assistant)

openai.types.beta.assistant.Assistant

## Add TruLens

In [16]:
from openai.resources.beta.assistants.assistants import Assistants
from openai.resources.beta.assistants.files import Files
from openai.resources.beta.threads.threads import Threads
from openai.resources.beta.threads.messages.messages import Messages
from openai.resources.beta.threads.runs.runs import Runs

from trulens_eval.tru_custom_app import instrument

instrument.method(Assistants, "create")
instrument.method(Assistants, "retrieve")
instrument.method(Threads, "create")
instrument.method(Threads, "retrieve")
instrument.method(Files, "create")
instrument.method(Files, "retrieve")
instrument.method(Messages, "create")
instrument.method(Messages, "retrieve")
instrument.method(Runs, "create")
instrument.method(Runs, "retrieve")

## Create a thread

In [17]:
thread = client.beta.threads.create()

In [18]:
from trulens_eval.tru_custom_app import TruCustomApp
tru_recorder = TruCustomApp(thread, "PG Essay Assistant")

AttributeError: 'str' object has no attribute 'items'

In [20]:
question = "how did pg grow up?"
message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=question
    )

run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
        instructions="Please answer any questions about Paul Graham."
    )

# results are asynchronous, need to wait until the run is completed
import time
while run.status in ['queued', 'in_progress', 'cancelling']:
    time.sleep(1)  # Wait for 1 second
    run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
    )

if run.status == 'completed': 
    messages = client.beta.threads.messages.list(
        thread_id=thread.id
        )
    response = messages.data[0].content[0].text.value  # Assuming the intention is to use or print this variable elsewhere
else:
    response = run.status  # Assuming the intention is to use or print this variable elsewhere


In [22]:
response

'Paul Graham grew up with a keen interest in both writing and programming, which were his main activities outside of school before college. His initial endeavor into programming involved writing programs on an IBM 1401, utilized by his school district for what was then referred to as "data processing"【7†source】.\n\nLater in his life, after college and during his time in graduate school and beyond, Graham explored the intersection of art and technology. At a certain point, he even considered becoming an artist and began taking art classes at Harvard. This period of exploration included planning to become an artist while also being deeply engaged in Lisp programming and working on "On Lisp." This multifaceted period of his life illustrated his eclectic interests and talents, spanning from computer science to the arts【9†source】.\n\nGraham\'s post-academic career involved a job at Interleaf, a company that had added a scripting language inspired by Emacs, making the scripting language a di

In [21]:
from trulens_eval import Tru

tru = Tru()

tru.get_leaderboard()

,latency,total_cost
app_id,,


In [ ]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.1.157:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>